From the TCR3d Database the TCR Alpha/Beta complexes are downloaded. These are TCR alpha and beta chains renumbered by AHO scheme. 
The naming shows the chain names (e.g. 3sxa_NO_aho.pdb)
Fist all chains are renamed as a and b.

In [ ]:
from Bio.PDB import PDBList, PDBIO, PDBParser
import os

pdbl = PDBList()

io = PDBIO()
parser = PDBParser()
directory="/home/lilian/TCR_Graphs/TCR3d_dataset_061023/tcr_structures/ab_chain"

for file in os.listdir(directory):
    filename = os.path.join(directory, file)
    if filename.endswith(".pdb"): 
        structure = parser.get_structure('file', filename)
        for model in structure:
            counter=0
            for chain in model:
                counter+=1
                old_name = chain.get_id()
                if counter == 1:
                    new_name = "A"
                elif counter == 2:
                    new_name = "B"
                #print(f"renaming chain {old_name} to {new_name}")
                chain.id = new_name
                
        io.set_structure(structure)
        io.save(filename)

In [1]:
!pip install graphein -q
!pip install "graphein[extras]" -q

In [2]:
def _load_esm_model(model_name: str = "esm1b_t33_650M_UR50S"):
    """
    Loads pre-trained FAIR ESM model from torch hub.
    """

    return torch.hub.load("facebookresearch/esm", model_name)

In [3]:
import numpy as np
def compute_esm_embedding(
    sequence: str,
    representation: str,
    model_name: str = "esm1b_t33_650M_UR50S",
    output_layer: int = 33,
) -> np.ndarray:
    """
    Computes sequence embedding using Pre-trained ESM model from FAIR
    """
    model, alphabet = _load_esm_model(model_name)
    batch_converter = alphabet.get_batch_converter()

    data = [
        ("protein1", sequence),
    ]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)

    # Extract per-residue representations (on CPU)
    with torch.no_grad():
        results = model(
            batch_tokens, repr_layers=[output_layer], return_contacts=True
        )
    token_representations = results["representations"][output_layer]

    if representation == "residue":
        return token_representations.numpy()

    # Generate per-sequence representations via averaging
    # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    elif representation == "sequence":
        sequence_representations = []
        for i, (_, seq) in enumerate(data):
            sequence_representations.append(
                token_representations[i, 1 : len(seq) + 1].mean(0)
            )
        return sequence_representations[0].numpy()

In [4]:
import logging
import os
from functools import lru_cache, partial
from pathlib import Path
from graphein.protein.features.sequence.utils import (
    compute_feature_over_chains,
    subset_by_node_feature_value)
import networkx as nx

def esm_residue_embedding(
    G: nx.Graph,
    model_name: str = "esm1b_t33_650M_UR50S",
    output_layer: int = 33,
) -> nx.Graph:
    """
    Computes ESM residue embeddings from a protein sequence and adds them to the graph.
    :param G: ``nx.Graph`` to add esm embedding to.
    :type G: nx.Graph
    :param model_name: Name of pre-trained model to use.
    :type model_name: str
    :param output_layer: index of output layer in pre-trained model.
    :type output_layer: int
    :return: ``nx.Graph`` with esm embedding feature added to nodes.
    :rtype: nx.Graph
    """

    for chain in G.graph["chain_ids"]:
        embedding = compute_esm_embedding(
            G.graph[f"sequence_{chain}"],
            representation="residue",
            model_name=model_name,
            output_layer=output_layer,
        )
        # remove start and end tokens from per-token residue embeddings
        embedding = embedding[0, 1:-1]
        subgraph = subset_by_node_feature_value(G, "chain_id", chain)

        for i, (n, d) in enumerate(subgraph.nodes(data=True)):
            G.nodes[n]["esm_embedding"] = embedding[i]

    return G

[10/25/23 16:57:06] WARNING  To use the Graphein submodule                                         ]8;id=112321;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/features/sequence/embeddings.py\embeddings.py]8;;\:]8;id=613104;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/features/sequence/embeddings.py#44\44]8;;\
                             graphein.protein.features.sequence.embeddings, you need to install:                   
                             biovec                                                                                
                             biovec cannot be installed via conda                                                  
                             Alternatively, you can install graphein with the extras:                              
                                                                                                                   
                             pip install graphein[extras]                                                          

                    WARNING  To use the Graphein submodule graphein.protein.visualisation, you  ]8;id=717896;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/visualisation.py\visualisation.py]8;;\:]8;id=319501;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/visualisation.py#35\35]8;;\
                             need to install: pytorch3d                                                            
                             To do so, use the following command: conda install -c pytorch3d                       
                             pytorch3d                                                                             

                    WARNING  To use the Graphein submodule graphein.protein.meshes, you need to        ]8;id=951991;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/meshes.py\meshes.py]8;;\:]8;id=985113;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/meshes.py#29\29]8;;\
                             install: pytorch3d                                                                    
                             To do so, use the following command: conda install -c pytorch3d pytorch3d             

In [5]:
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.edges.distance import add_aromatic_interactions, add_cation_pi_interactions, add_hydrophobic_interactions, add_ionic_interactions
from functools import partial
from graphein.protein.edges.distance import add_distance_threshold
from graphein.protein.visualisation import plotly_protein_structure_graph
from graphein.protein.features.sequence.embeddings import esm_sequence_embedding, biovec_sequence_embedding

config = ProteinGraphConfig(edge_construction_functions=[add_aromatic_interactions,
                                                         add_cation_pi_interactions,
                                                         add_hydrophobic_interactions,
                                                         add_ionic_interactions,
                                                         partial(add_distance_threshold, long_interaction_threshold=5, threshold=10.)],
                            graph_metadata_functions=[esm_sequence_embedding,esm_residue_embedding])
#node_metadata_functions=[esm_residue_embedding]


In [6]:
import torch
from graphein.ml import ProteinGraphDataset
import graphein.protein as gp

#directory="/home/lilian/TCR_Graphs/TCR3d_dataset_061023/tcr_structures/ab_chain"
directory="/home/lilian/TCR_Graphs/Novozyme_data/AF_WT/pdb"
pdb_list=[]
for file in os.listdir(directory):
    filename = os.path.join(directory, file)
    pdb_list.append(filename)


In [7]:
#For local PDB files
from graphein.protein.graphs import construct_graph
from graphein.ml import ProteinGraphListDataset, GraphFormatConvertor
import graphein.protein as gp
graphs=[]
failed_graphs=[]
for i in pdb_list:
    print(i)
    try:
        g=construct_graph(config=config, path=i, verbose=False)
        graphs.append(g)
    except:
        print("error")
        failed_graphs.append(i)
        
        continue
convertor = GraphFormatConvertor(src_format="nx", dst_format="pyg")
graphs = [convertor(g) for g in graphs]

# Create dataset
ds = ProteinGraphListDataset(root="./", data_list=graphs, name="Novo_WT")

/home/lilian/TCR_Graphs/Novozyme_data/AF_WT/pdb/AF-A0A829LLI5-F1-model_v3.pdb


/home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/graphs.py:161: FutureWarning: The default value of regex will change from True to False in a future version.
  df["node_id"] = df["node_id"].str.replace(":$", "")
/home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/graphs.py:164: FutureWarning: The default value of regex will change from True to False in a future version.
  df["node_id"] = df["node_id"].str.replace(":$", "")


[10/25/23 16:58:05] DEBUG    Deprotonating protein. This removes H atoms from the pdb_df dataframe    ]8;id=630420;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=143516;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/graphs.py#187\187]8;;\

                    DEBUG    Detected 663 total nodes                                                 ]8;id=861747;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=371996;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/graphs.py#431\431]8;;\

                    DEBUG    Reading meiler embeddings from:                                       ]8;id=999417;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/features/nodes/amino_acid.py\amino_acid.py]8;;\:]8;id=143205;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/features/nodes/amino_acid.py#57\57]8;;\
                             /home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-pa                 
                             ckages/graphein/protein/features/nodes/meiler_embeddings.csv                          

...

                    INFO     Found 1049 hydrophobic interactions.                                  ]8;id=415001;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/edges/distance.py\distance.py]8;;\:]8;id=34465;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/edges/distance.py#1343\1343]8;;\

                    INFO     Found 3592 ionic interactions.                                        ]8;id=739152;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/edges/distance.py\distance.py]8;;\:]8;id=727416;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/edges/distance.py#1343\1343]8;;\

                    INFO     Found: 9129 distance edges                                             ]8;id=696963;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/edges/distance.py\distance.py]8;;\:]8;id=741223;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/edges/distance.py#963\963]8;;\

[10/25/23 17:18:25] INFO     Added 5456 distance edges. (3673            removed by LIN)            ]8;id=306261;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/edges/distance.py\distance.py]8;;\:]8;id=49590;file:///home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/graphein/protein/edges/distance.py#982\982]8;;\

Using cache found in /home/lilian/.cache/torch/hub/facebookresearch_esm_main


In [ ]:
print(len(failed_graphs))
print(len(graphs))

106
394


In [ ]:
import numpy as np
np.save('Novo_WT',ds, allow_pickle =True)

/home/lilian/miniconda3/envs/py311-pyg-cuda118/lib/python3.11/site-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [9]:
ds= np.load('TCCR3d_data.npy', allow_pickle =True)

In [10]:
from graphein.ml import ProteinGraphListDataset, GraphFormatConvertor
ds_pyg = ProteinGraphListDataset(root="./", data_list=ds, name="TCR3d_abchains")